<a href="https://colab.research.google.com/github/SBrouck/data-automation-booking-airbnb/blob/main/DataExtractGithub.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# ---------------------------
# INSTALL REQUIRED PACKAGES
# ---------------------------
!pip install ics openpyxl

# ---------------------------
# IMPORTS
# ---------------------------
import pandas as pd
import requests
from ics import Calendar
from datetime import datetime
from google.colab import files

# ---------------------------
# UPLOAD USER FILES
# ---------------------------
print("Veuillez importer un fichier CSV Booking et un fichier Excel Airbnb (.xlsx)")
uploaded = files.upload()

booking_file = [f for f in uploaded if f.endswith('.csv')][0]
airbnb_file = [f for f in uploaded if f.endswith('.xlsx')][0]

# ---------------------------
# LOAD BOOKING DATA
# ---------------------------
booking_df = pd.read_csv(booking_file)
booking_df = booking_df[[
    "Listing", "Guest name", "Start date", "End date", "# of nights", "Earnings"
]].copy()
booking_df["Start date"] = pd.to_datetime(booking_df["Start date"])
booking_df["End date"] = pd.to_datetime(booking_df["End date"])
booking_df["Platform"] = "Booking"

# ---------------------------
# LOAD AIRBNB DATA
# ---------------------------
airbnb_df = pd.read_excel(airbnb_file)
airbnb_df = airbnb_df[[
    "Nom de l'établissement", "Nom du client", "Arrivée", "Départ", "Paiement total"
]].copy()
airbnb_df.columns = ["Listing", "Guest name", "Start date", "End date", "Earnings"]
airbnb_df["Start date"] = pd.to_datetime(airbnb_df["Start date"], errors="coerce", dayfirst=True)
airbnb_df["End date"] = pd.to_datetime(airbnb_df["End date"], errors="coerce", dayfirst=True)
airbnb_df["# of nights"] = (airbnb_df["End date"] - airbnb_df["Start date"]).dt.days
airbnb_df["Platform"] = "Airbnb"

# ---------------------------
# MERGE PAST RESERVATIONS
# ---------------------------
past_reservations = pd.concat([airbnb_df, booking_df], ignore_index=True)
past_reservations = past_reservations.sort_values(by="Start date")

# ---------------------------
# DEFINE ICAL LINKS (TO BE MODIFIED BY USER)
# ---------------------------
ICAL_LINKS = [
    {
        "name": "Example Apartment",
        "url": "https://www.airbnb.com/calendar/ical/EXAMPLE.ics?s=EXAMPLEKEY"
    },
    # Add more listings here
]

# ---------------------------
# EXTRACT FUTURE RESERVATIONS FROM ICAL
# ---------------------------
future_data = []

for listing in ICAL_LINKS:
    try:
        r = requests.get(listing["url"])
        calendar = Calendar(r.text)
        for event in calendar.events:
            if event.begin.date() >= datetime.today().date():
                future_data.append({
                    "Listing": listing["name"],
                    "Start date": event.begin.date(),
                    "End date": event.end.date(),
                    "Nights": (event.end.date() - event.begin.date()).days,
                    "Source": "iCal"
                })
    except Exception as e:
        print(f"Erreur avec {listing['name']}: {e}")

calendar_df = pd.DataFrame(future_data)
calendar_df = calendar_df.sort_values(by="Start date")

# ---------------------------
# EXPORT FINAL FILE WITH 2 SHEETS
# ---------------------------
output_filename = "reservation_summary.xlsx"

with pd.ExcelWriter(output_filename, engine="openpyxl") as writer:
    past_reservations.to_excel(writer, sheet_name="Revenus_passés", index=False)
    calendar_df.to_excel(writer, sheet_name="Calendrier_a_venir", index=False)

files.download(output_filename)
